In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [16]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lin(x)
        return torch.sigmoid(x)

In [17]:
device=0
log_steps=1
num_layers=1
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [18]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [19]:
adj_t = data.adj_t.to(device)

In [20]:
torch.manual_seed(12345)

In [21]:
model = SAGE(hidden_channels, hidden_channels,
             dropout).to(device)

In [22]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [23]:
predictor = LinkPredictor(hidden_channels, 1).to(device)

In [24]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [25]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [26]:
model.reset_parameters()
predictor.reset_parameters()

In [27]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [28]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [29]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 0.9764
Epoch: 04, Loss: 0.8561
Epoch: 06, Loss: 0.7493
Epoch: 08, Loss: 0.6867
Epoch: 10, Loss: 0.6233
Epoch: 12, Loss: 0.5675
Epoch: 14, Loss: 0.5217
Epoch: 16, Loss: 0.4715
Epoch: 18, Loss: 0.4262
Epoch: 20, Loss: 0.3915
Epoch: 22, Loss: 0.3562
Epoch: 24, Loss: 0.3321
Epoch: 26, Loss: 0.3077
Epoch: 28, Loss: 0.2861
Epoch: 30, Loss: 0.2638
Epoch: 32, Loss: 0.2437
Epoch: 34, Loss: 0.2277
Epoch: 36, Loss: 0.2130
Epoch: 38, Loss: 0.1990
Epoch: 40, Loss: 0.1869
Epoch: 42, Loss: 0.1767
Epoch: 44, Loss: 0.1665
Epoch: 46, Loss: 0.1571
Epoch: 48, Loss: 0.1493
Epoch: 50, Loss: 0.1416
Epoch: 52, Loss: 0.1349
Epoch: 54, Loss: 0.1285
Epoch: 56, Loss: 0.1227
Epoch: 58, Loss: 0.1166
Epoch: 60, Loss: 0.1110
Epoch: 62, Loss: 0.1058
Epoch: 64, Loss: 0.1016
Epoch: 66, Loss: 0.0982
Epoch: 68, Loss: 0.0942
Epoch: 70, Loss: 0.0903
Epoch: 72, Loss: 0.0870
Epoch: 74, Loss: 0.0842
Epoch: 76, Loss: 0.0813
Epoch: 78, Loss: 0.0785
Epoch: 80, Loss: 0.0755
Epoch: 82, Loss: 0.0728
Epoch: 84, Loss:

In [30]:
predictor.lin.weight

Parameter containing:
tensor([[1.1896, 1.4459, 1.1338, 1.2402, 1.0555, 1.1475, 1.3328, 1.1936, 1.0301,
         1.0774, 1.1115, 1.1587, 1.0725, 1.1604, 1.0804, 1.0288, 1.1297, 1.2014,
         1.2258, 1.2762, 1.1281, 1.0166, 1.3971, 1.1980, 1.0943, 0.9973, 1.0443,
         1.1536, 1.2125, 1.1489, 1.1733, 1.1425, 1.2231, 1.1866, 1.0806, 1.0404,
         1.1260, 1.0387, 1.1598, 0.9950, 1.1039, 1.1471, 1.2440, 1.3079, 0.9607,
         1.1720, 1.0738, 1.0588, 1.1591, 1.1847]], device='cuda:0',
       requires_grad=True)

In [31]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
  )
)

In [32]:
emb.weight

Parameter containing:
tensor([[ 0.2915, -0.5625,  0.1320,  ..., -0.4136,  0.0230,  0.1214],
        [-0.0397,  0.1172,  0.1175,  ...,  0.1847,  0.0821, -0.0307],
        [-0.1959,  0.1398,  0.1372,  ..., -0.2343, -0.1878,  0.0625],
        ...,
        [ 0.1398,  0.1138, -0.1345,  ..., -0.0459,  0.2057,  0.1261],
        [ 0.0333, -0.2890, -0.0514,  ..., -0.2688,  0.0968, -0.0481],
        [-0.1300,  0.1349,  0.0869,  ...,  0.0796,  0.0738, -0.0700]],
       device='cuda:0', requires_grad=True)

In [33]:
h = model(emb.weight, adj_t)

In [34]:
h

tensor([[ 0.4908,  0.6662, -0.5734,  ...,  0.6578,  0.7374,  0.8501],
        [ 0.2486,  0.3619,  0.0535,  ...,  0.1343, -0.2551, -0.0771],
        [ 0.5583, -0.5166,  0.6699,  ..., -0.6748,  0.3060, -0.5040],
        ...,
        [-0.4788, -0.2193,  0.5848,  ...,  0.2071, -0.4606,  0.3084],
        [ 0.4071, -0.0937,  0.0074,  ...,  0.6124,  0.3130,  0.5037],
        [-0.1100,  0.0197,  0.0495,  ..., -0.1894, -0.2193,  0.5262]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [35]:
predictor(h[0], h[411])

tensor([0.9980], device='cuda:0', grad_fn=<SigmoidBackward>)

In [36]:
predictor(h[0], h[2])

tensor([0.0588], device='cuda:0', grad_fn=<SigmoidBackward>)

In [37]:
(h[0] * h[2]).sum()

tensor(1.9060, device='cuda:0', grad_fn=<SumBackward0>)

In [38]:
(h[0] * h[411]).sum()

tensor(9.6488, device='cuda:0', grad_fn=<SumBackward0>)

In [39]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_39_h.npy', h.cpu().detach().numpy())
torch.save(model, '../../models/graphsage_link_pred/39_model.pt')
torch.save(emb, '../../models/graphsage_link_pred/39_embedding.pt')
torch.save(predictor, '../../models/graphsage_link_pred/39_predictor.pt')